In [1]:
import os
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras.preprocessing import image
from keras import Sequential
from keras.layers import Dense,Activation, Conv2D, Flatten, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.models import Model
from keras.metrics import categorical_crossentropy
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,max_error, explained_variance_score,median_absolute_error, accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import random
import seaborn as sns
import matplotlib.pyplot as plt
from skimage import *
import cv2
import math
import joblib
from scipy import stats
import gc


img_w= 256
img_h=256

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path= '/content/drive/MyDrive/Colab Notebooks/primary_small/train'
valid_path= '/content/drive/MyDrive/Colab Notebooks/primary_small/valid'
test_path= '/content/drive/MyDrive/Colab Notebooks/primary_small/test'
classes = ["brightpixel", "narrowband","narrowbanddrd","noise","squarepulsednarrowband","squiggle","squigglesquarepulsednarrowband"]

In [4]:
class ImageData:

    def __init__(self, filepath):
        self.filepath= filepath
        self.X_data= []
        self.y_data= []
        self.X_gray= []
        self.X_binary=[]
        self.classes=[]
        self.y_categorical= []
           
            
            
    def split_and_resize(self): 

        self.classes= sorted(os.listdir(self.filepath))

        for c in range(len(self.classes)):
            path = os.path.join(self.filepath, self.classes[c])
            signal_type= self.classes[c]
            
            for i in os.listdir(path):
                img= cv2.imread(os.path.join(path, i))
                img= cv2.resize(img, (256, 256))
                
                self.X_data.append(img)
                self.y_data.append(signal_type)
                
        return np.array(self.X_data), np.array(self.y_data)

    
    def grayscale(self):

        for i in range(len(self.X_data)):
            x= cv2.cvtColor(self.X_data[i], cv2.COLOR_BGR2GRAY)
            self.X_gray.append(x)

            
    
    def create_binary_image(self):
        self.grayscale()
        for i in range(len(self.X_gray)):
            img= np.where(self.X_gray[i]>100,1,0)
            self.X_binary.append(img)
        return np.array(self.X_binary)
            
    
    def create_categorical_label(self):
        self.y_categorical = to_categorical([self.classes.index(i) for i in self.y_data])
        return self.y_categorical
    

In [5]:
train= ImageData(train_path)
X_tr, y_tr= train.split_and_resize()

classes= train.classes

In [6]:
valid= ImageData(valid_path)
X_val, y_val= valid.split_and_resize()

In [7]:
test= ImageData(test_path)
X_te, y_te= test.split_and_resize()

In [8]:
y_te_C= test.create_categorical_label()
y_val_C= valid.create_categorical_label()
y_tr_C= train.create_categorical_label()

In [ ]:
def use_efficient_net(model_type='B0'):
    from tensorflow.keras.optimizers import RMSprop
    from keras.models import Model
    import efficientnet.tfkeras as efn
    if model_type=='B0':
        efn_model = efn.EfficientNetB0(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    else:
        efn_model = efn.EfficientNetB7(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    for layer in efn_model.layers:
        layer.trainable = False
    #
    x = efn_model.output
    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    # Add a final sigmoid layer with 1 node for classification output
    predictions = Dense(1, activation="sigmoid")(x)
    efficient_net = Model(efn_model.input,predictions)

    efficient_net.compile(RMSprop(learning_rate=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])
    return efficient_net
efficient_net=use_efficient_net('B0')
efficient_net.summary()


In [ ]:
efficient_net.evaluate(test_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# create data
x=[]
for i in range(1,101):
    x.append(i)

# plot lines
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.plot(x, effb0_history.history['accuracy'], label = "EfficientNet-B0 Model Training", linestyle="-")
plt.plot(x, effb0_history.history['val_accuracy'], label = "EfficientNet-B0 Model Validation", linestyle="-")
plt.legend()
plt.show()

In [ ]:
accuracy = history.history['accuracy']
accuracy_val= history.history['val_accuracy']
epochs= range(1,41)
plt.plot(epochs, accuracy, 'g', label='Training accuracy')
plt.plot(epochs, accuracy_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
preds= MODEL.predict(X_te)
preds= np.argmax(preds, axis=1)

In [ ]:
new_list=[]
for i in range(len(preds)):
    if preds[i]==0:
        new_list.append('brightpixel')
    elif preds[i]==1:
        new_list.append('narrowband')
    elif preds[i]==2:
        new_list.append('narrowbanddrd')
    elif preds[i]==3:
        new_list.append('noise')
    elif preds[i]==4:
        new_list.append('squarepulsednarrowband')
    elif preds[i]==5:
        new_list.append('squiggle')
    elif preds[i]==6:
        new_list.append('squigglesquarepulsednarrowband')
        
new_preds= np.array(new_list)

In [ ]:
accuracy_score(y_te, new_preds)